# 01 Spark Dataframes

---

## 1.1 Setting Up A SparkSession

To use PySpark, we must first set up a `SparkSession`. A `SparkSession` is an API through which Python code gets translated to Spark code. To start a `SparkSession`, we load in the `SparkSession` object from the `pyspark.sql` module and running some boilerplate:

In [5]:
from pyspark.sql import SparkSession

# build a SparkSession
spark = SparkSession.builder.appName("Spark Basics").getOrCreate()

<br>

---

<br>

## 1.2 Reading Data

Once a `SparkSession` has been set up, we can begin reading in data. This can be done in multiple different ways depending on the context of where we are running our code. For example:

1) If we are running our code on a local machine, we can read in flat files like `.csv`, `.txt`, `.JSON`, etc. directly into memory.

2) If we are on Databricks, we can query an existing table in the data warehouse using SQL syntax: `spark.sql("SELECT * FROM mytable")`

This notebook will load data from a local machine, which can be done by using the various `read.<file extension>` functions.

In [6]:
df_people = spark.read.json("../course_materials/Spark_Dataframes/people.json")
df_sales = spark.read.csv("../course_materials/Spark_Dataframes/sales_info.csv")

It is also possible to first read in a flat file as a Pandas dataframe, then convert it into a Spark dataframe. This can be done using the `createDataFrame()` method from the `SparkSession` object. This is convenient if Pandas has a specific functionality that we want to take advantage (such as inferring schema automatically from `.csv` files).

In [7]:
import pandas as pd

# read
df_sales = pd.read_csv("../course_materials/Spark_Dataframes/sales_info.csv")

df_sales = spark.createDataFrame(df_sales)

In [8]:
df_sales.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: long (nullable = true)



<br>

---

<br>

## 1.3 Spark Dataframes

Spark will store data in so-called **Spark Dataframes**. Unlike PANDAS dataframes, operations on Spark dataframes are meant to be executed on distributed compute / compute clusters (e.g. Databricks) and are meant to be manipulated using the Spark language. Consequently, this means that Spark dataframes are not compatible with standard python functions like `print()`. Instead, we will generally need to methods specific to Spark dataframe objects in order to manipulate them correctly.

In [6]:
# trying to print a spark dataframe with print()
# won't actually show the dataframe
print(df_people)

DataFrame[age: bigint, name: string]


In [21]:
# we can "print" a Spark dataframe
# by calling the show() method
df_people.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



<br>

---

<br>

## 1.4 Schemas

The structure of a Spark dataframe is defined by its `schema`. Just like SQL, a **schema** is a dictionary that assigns each column a specific data typing and additional options (e.g. can the column contain nulls?)

In [8]:
# the schema of a dataframe can be viewed with the
# printSchema() method
df_people.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



Spark dataframes come with a `columns` attribute, which is just a python list of the column names:

In [11]:
# look at the columns attribute;
# note that this attribute is a just python list
# so it will print as expected
df_people.columns

['age', 'name']

<br>

---

<br>

## 1.5 Structure Fields and Editing Schemas

The schema of a Spark dataframe changed, which is useful if the `read.<file type>` functions don't behave like we want them to. In order to change the schema, we need to create a new `StructType` object. Again, because Spark dataframes have to be manipulated using Spark, the data types themselves must be loaded from a PySpark modueld `pyspark.sql.types`.

In PySpark, a schema is just a **list of `StructField`** objects and a `StructField` object is specified by the following datum:

1) The name of the column
2) The type of the column
3) Can the column contain nulls?

Once this data is specified into a list of `StructField` objects, we can construct a `StructType` object which translates the datum into the necessary spark code to change the schema.

In [22]:
from pyspark.sql.types import StructField, StructType, StringType, FloatType

# specify a "new" schema for df_people where:
# "age" = float (nulls are permitted)
# "name" = string (nulls not permitted)
# Note that the order of the fields must match the column order in the 
# underlying file
new_fields = [
    StructField("age", FloatType(), nullable = True),
    StructField("name", StringType(), nullable = False),
]

# package the new fields into a new schema
new_schema = StructType(fields = new_fields)

# use this schema when reading in data files
df_new_people = spark.read.json("../course_materials/Spark_Dataframes/sales_info.csv", schema = new_schema)

In [23]:
# check that the new schema is being used
df_new_people.printSchema()

root
 |-- age: float (nullable = true)
 |-- name: string (nullable = true)



<br>

---

<br>

## 1.6 Column Objects

Typically, we manipulate Spark dataframes by manipulating the columns. Columns for a Spark dataframe are implemented as Spark `Column` objects and must be interfaced with using Spark methods:

In [25]:
# A column in a spark dataframe is a Spark Column object
type(df_people["age"])

pyspark.sql.column.Column

### The `select()` Method

The `select()` method will collect together multiple Column objects and return them as a Spark dataframe. This allows us to then use dataframe methods on the columns by first using `select()`.

In [29]:
# select using SQL syntax; 
# we just pass the column name(s)
df_people.select("age").show()

# select using "dataframe" syntax;
# we specify the column object(s) to select
df_people.select( df_people["age"] ).show()

+----+
| age|
+----+
|NULL|
|  30|
|  19|
+----+

+----+
| age|
+----+
|NULL|
|  30|
|  19|
+----+



<br>

---

<br>

## 1.7 Row Objects

Spark dataframes can be thought of as a list of Row objects. Accessing a specific value inside of a dataframe will typically require us to:

1) Return the dataframe as a list of Row objects

2) In the specific Row, access the specific column we need

### The `collect()` Method

To return the dataframe as a list of rows, we use the `collect()` method. We can then acccess specific values by accessing the Row objects themselves. Note that a row object behaves like a "named list" so values within a Row object can be referenced by index (column number) or by name (column name).

In [33]:
# return the dataframe as a list of Row objects
df_people.collect()

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin')]

In [40]:
# get the age of Justin
df_people.collect()[2]["age"]

19

<br>

---

<br>

## 1.8 SQL Queries

Spark dataframes can also be temporarily stored in memory as SQL tables, allowing us to query them using SQL. To do this, we typically:

1) Create a temporary view using the spark dataframe
2) Execute a SQL query against the view

In [4]:
# create a temporary view using the dataframe
df_people.createOrReplaceTempView("people")

In [5]:
# query the view using sql
spark.sql("SELECT * FROM people WHERE age = 30").show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



<br>

---

<br>